In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import lightgbm as lgb

In [ ]:
import riiideducation
env = riiideducation.make_env()
iter_test = env.iter_test()

# Data

In [ ]:
train_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv', nrows=10**6)

print(train_df.shape)
train_df.head()

In [ ]:
test_df = pd.read_csv("/kaggle/input/riiid-test-answer-prediction/example_test.csv")

print(test_df.shape)
test_df.head()

In [ ]:
submission_df = pd.read_csv("/kaggle/input/riiid-test-answer-prediction/example_sample_submission.csv")

print(submission_df.shape)
submission_df.head()

# Preprocess

In [ ]:
train_df['prior_question_had_explanation'].fillna(value=False, inplace=True)
train_df.fillna(value=0.5, inplace=True)

print(train_df.isnull().sum().sum())

In [ ]:
def code_transform(x):

    if x == True:
        y = 0
    elif x == False:
        y = 1

    return y

train_df['prior_question_had_explanation'] = train_df['prior_question_had_explanation'].apply(lambda x: code_transform(x))
train_df['prior_question_had_explanation'] = train_df['prior_question_had_explanation'].astype('int')

print(train_df.dtypes)
train_df.head()

# Modeling

In [ ]:
features = ['timestamp','user_id', 'content_id', 'prior_question_elapsed_time','prior_question_had_explanation']
target = 'answered_correctly'

print(train_df[features].shape)
print(train_df[target].shape)

In [ ]:
print(train_df[features].dtypes)
train_df[features].head()

In [ ]:
params = {'objective': 'binary',
          'seed': 2020,
          'learning_rate': 0.1}

model = lgb.LGBMClassifier(**params)
model.fit(train_df[features], train_df[target])

In [ ]:
for (test_df, sample_prediction) in iter_test:
    
    test_df['prior_question_had_explanation'].fillna(value=False, inplace=True)
    test_df.fillna(value=0.5, inplace=True)
    
    test_df['prior_question_had_explanation'] = test_df['prior_question_had_explanation'].apply(lambda x: code_transform(x))
    test_df['prior_question_had_explanation'] = test_df['prior_question_had_explanation'].astype('int')
    
    test_df['answered_correctly'] = model.predict_proba(test_df[features])[:,1]
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])

Kaggle Submit Score: 0.414